# Notebook 5.1 - Curation-Keywords

This notebook implements the workflow defined in:

[Curating keywords](https://gitlab.gwdg.de/sshoc/marketplace-curation/-/issues/1#note_71056) GitLab issue

The notebook works as follows:

0. Imports external libraries and loads the MP dataset and the google sheet
2. Updates keywords on MP as follows:
    1. Looks for vocabulary terms with the value from the column *Keyword to map*
    2. Looks for the term in the column *Map to*
    3. Goes through all MP items to look for the *keywords-to-map* in the keyword-dynamic-property
    4. Replaces the  the *keywords-to-map* in the MP dataset.


## 0 Requirements to run this notebook

This section gives all the relevant information to "interact" with the MP data.

### 0.1 libraries
*There are a number of external libraries needed to run the notebook* 

*Furthermore, a dedicated SSH Open Marketplace library - sshmarketplacelib - with customised functions has been created and can be imported using the python import commands.* 

*Below the libraries import needed to run this notebook*

In [1]:
import numpy as np
import pandas as pd
import requests
#import the MarketPlace Library 
from sshmarketplacelib import MPData as mpd
from sshmarketplacelib import  eval as eva, helper as hel

### 0.2 Get the data



Get the MarketPlace dataset

In [2]:
mpdata = mpd()
df_tool_flat =mpdata.getMPItems ("toolsandservices", True)
df_publication_flat =mpdata.getMPItems ("publications", True)
df_trainingmaterials_flat =mpdata.getMPItems ("trainingmaterials", True)
df_workflows_flat =mpdata.getMPItems ("workflows", True)
df_datasets_flat =mpdata.getMPItems ("datasets", True)

getting data from local repository...
getting data from local repository...
getting data from local repository...
getting data from local repository...
getting data from local repository...


Get the list of keywords from the [gsheet](https://docs.google.com/spreadsheets/d/1-Oh9_SxIhfMAT6KNJrMf4LetCpy5s1fHZEyTL__TUVA/edit#gid=0)

In [3]:
sheet_id = '1-Oh9_SxIhfMAT6KNJrMf4LetCpy5s1fHZEyTL__TUVA'
sheet_name = 'Mappings'
url = f'https://docs.google.com/spreadsheets/d/{sheet_id}/gviz/tq?tqx=out:csv&sheet={sheet_name}'
df_keywords=pd.read_csv(url)

The function *getMPConcepts()* is a custum function that uses the API entry: 

GET https://marketplace-api.sshopencloud.eu/api/concept-search?perpage=100&q=URI

to get all the *concepts* from the MarketPlace dataset. 

**Note that executing this function may require some time, currently 14995 records are returned**



In [4]:
df_concepts=mpdata.getMPConcepts()

In [5]:
utils=hel.Util()
resultfields=['persistentId', 'MPUrl', 'category', 'label', 'type.code', 'type.label', 'concept.code', 'concept.label', 'concept.uri', 'concept.vocabulary.scheme']
udf_alprop=utils.getAllPropertiesBySources()
udf_alprop=udf_alprop.loc[ : ,resultfields]

A few lines of the gsheet

In [7]:
df_keywords.head()

,Keyword to map,Map to,Comment,Discussion
0,Linguistics,https://vocabs.acdh.oeaw.ac.at/oefosdiscipline...,Sprach- und Literaturwissenschaften,NaN
1,History,https://vocabs.acdh.oeaw.ac.at/oefosdiscipline...,"Geschichte, Archäologie",NaN
2,Literature,https://vocabs.acdh.oeaw.ac.at/oefosdiscipline...,Sprach- und Literaturwissenschaften,NaN
3,Video,https://vocabs.sshopencloud.eu/vocabularies/me...,video,NaN
4,Text,https://vocabs.sshopencloud.eu/vocabularies/me...,text,NaN


### 1 Update keywords

The function *getMPKeywordProperies(mKey)* used below is a custum function that uses the API entry 

    GET https://marketplace-api.sshopencloud.eu/api/concept-search?types=keyword&q=VALUE

and returns the vocabulary terms for *mKey*.  

The returned dataset is filtered to individuate those values coming from the vocabulary *sshoc-keyword* (vocabulary[code]=sshoc-keyword), then the resulting dataset is shown.


In [ ]:


selectedItems=pd.DataFrame()
for rown, row in df_keywords.iterrows():
    myKey=df_keywords.iloc[rown]['Keyword to map']
    #df_vocterms=pd.DataFrame()
    df_vocterms=mpdata.getMPKeywordProperties(myKey).loc[:]
    print(f'\n***** checked {myKey}')
    if (df_vocterms.empty):
        print (f"#####  No vocabulary terms found for {myKey}")
        continue;
    df_vocterms=df_vocterms.loc[(df_vocterms.vocabulary=={'code': 'sshoc-keyword'}), ]
    #The set of vocabulary terms is filterd to individuate those with the exact match (case-insensitive) 
    df_vocterms=df_vocterms.loc[(df_vocterms.code).str.lower()==myKey.lower()]

    #Search for the one concept that has in the uri column the value of the Map to. There should be only one such concept.
  
    df_mapto=df_concepts.loc[(df_concepts.uri==df_keywords.iloc[rown]['Map to']),]

    #Filter the dataset using the Keyword to Map

    df_items=udf_alprop.loc[(udf_alprop['concept.label'].str.lower()==myKey.lower()), ]#& (udf_alprop['type.code'].str.lower()=='keyword')]
    
    if (df_items.empty):
        print (f"%%%%%  No Items found for {myKey}")
        continue;
    print (f'&&&&&  Found as {df_items.iloc[0]["type.code"]}')
    #update the MP
    jsonConcept={}
    jsonConceptVal={}
    jsonConceptVal["code"]=df_mapto.iloc[0].code
    jsonConceptVal["vocabulary"]=df_mapto.iloc[0].vocabulary
    jsonConceptVal["uri"]=df_mapto.iloc[0].uri
    #jsonConcept
    attrList={}
    attrList["type"]=df_mapto.iloc[0].types[0]
    attrList["concept"]=jsonConceptVal
    filterList={}
    filterList["concept"]=myKey.lower()
   
    df_items.loc[ : ,'updateList']=[attrList for _ in range(df_items.shape[0])]
    
    df_items.loc[ : ,'filterList']=[filterList for _ in range(df_items.shape[0])]
   
    selectedItems=selectedItems.append(df_items)
    

attrList={}
filterList={}
mpdata.updateItemsProperties(selectedItems)


In [ ]:
selectedItems.count()